In [1]:
## basic libraries → data exploration
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
import os
# from google.colab import drive

from tensorflow.keras.applications.resnet import ResNet152
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model

In [ ]:
# drive.mount('content')

In [2]:
# defining data path
train_path = '../datasets/train/'
test_path = '../datasets/validate/'

In [3]:
## loading the data on drive
# df_test_labels = pd.read_csv('content/MyDrive/Thesis_Smart_parking_222/Dataset_Hung/class_names_dvm.csv')
# train_car = glob("/content/MyDrive/Thesis_Smart_parking_222/Dataset_Hung/train/*/*")
# test_car = glob("/content/MyDrive/Thesis_Smart_parking_222/Dataset_Hung/test/*/*")

## loading the data on local
df_test_labels = pd.read_csv('../datasets/class_names_dvm.csv')
train_car = glob(f"{train_path}*/*")
test_car = glob(f"{test_path}*/*")

In [4]:
class_names = list(df_test_labels["Cars"].unique())
class_names

['Aston_Martin~DB11~2017',
 'Aston_Martin~DB11~2019',
 'Aston_Martin~DBS~2019',
 'Audi~A1~2015',
 'Audi~A1~2016',
 'Audi~A1~2017',
 'Audi~A1~2018',
 'Audi~A1~2019',
 'Audi~A3 Cabriolet~2015',
 'Audi~A3 Cabriolet~2016',
 'Audi~A3 Cabriolet~2017',
 'Audi~A3~2015',
 'Audi~A3~2016',
 'Audi~A3~2017',
 'Audi~A3~2018',
 'Audi~A4 Allroad~2015',
 'Audi~A4 Allroad~2017',
 'Audi~A4 Avant~2015',
 'Audi~A4 Avant~2016',
 'Audi~A4 Avant~2017',
 'Audi~A4 Avant~2018',
 'Audi~A4 Avant~2019',
 'Audi~A4~2019',
 'Audi~A5 Cabriolet~2015',
 'Audi~A5 Cabriolet~2017',
 'Audi~A5 Cabriolet~2018',
 'Audi~A5 Cabriolet~2019',
 'Audi~A5~2015',
 'Audi~A5~2019',
 'Audi~A6 Allroad~2015',
 'Audi~A6 Allroad~2016',
 'Audi~A6 Allroad~2017',
 'Audi~A6 Avant~2015',
 'Audi~A6 Avant~2016',
 'Audi~A6 Avant~2017',
 'Audi~A6 Avant~2018',
 'Audi~A6 Saloon~2015',
 'Audi~A6 Saloon~2016',
 'Audi~A6 Saloon~2017',
 'Audi~A6 Saloon~2018',
 'Audi~A6 Unspecified~2016',
 'Audi~A6 Unspecified~2017',
 'Audi~A6~2019',
 'Audi~A7~2015',
 'Audi~

In [5]:
## setting up some parameters for data augmentation
img_width, img_height = 224, 224
num_channels = 3
train_samples = len(train_car)
test_samples = len(test_car)
num_of_class_names = len(class_names)
batch_size = 16
epochs = 50

In [6]:
## performing augmentation on the training data
train_datagen = ImageDataGenerator(
  zoom_range=0.2,
  rescale=1./255,
  rotation_range = 20,
  horizontal_flip=True
)

valid_datagen = ImageDataGenerator()

In [7]:
## converting data to a tf.data.Dataset object
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

valid_generator = valid_datagen.flow_from_directory(
    test_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


# Build ResNet152 Model

In [8]:
## load the ResNet152 model
model = ResNet152(
  include_top=False,
  weights='imagenet',
  input_tensor=None,
  input_shape=(img_width, img_height, 3),
  classes=num_of_class_names,
)

In [9]:
## add some extra layers
x = model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(
  units=1024,
  activation='relu',
  kernel_regularizer=regularizers.l2(0.01), 
  kernel_initializer='random_uniform',
  bias_initializer='zeros'
)(x)
x = Dense(
  units=1024,
  activation='relu',
  kernel_regularizer=regularizers.l2(0.01), 
  kernel_initializer='random_uniform',
  bias_initializer='zeros'
)(x)
output = Dense(units=num_of_class_names, activation='softmax')(x)

## create the extended model
tf_model = Model(inputs=model.input, outputs=output)

# freeze all layers excepts batchnorm layers
for layer in model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False


In [10]:
model.summary()

Model: "resnet152"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                          

In [64]:
## compile the model, define optimizer and the loss function
opt = Adam(learning_rate=0.01)

tf_model.compile(
  loss='categorical_crossentropy',
  optimizer=opt, metrics=['accuracy']
)

In [65]:
checkpoint_path = "../model_resnet152/cp_resnet152_model.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = ModelCheckpoint(
  filepath=checkpoint_path,
  save_weights_only=True,
  verbose=1
)

In [66]:
# tf_model.load_weights(checkpoint_path)

In [ ]:
training_history = tf_model.fit(
  train_generator,
  steps_per_epoch=len(train_generator),
  validation_data=valid_generator,
  validation_steps=(len(valid_generator) / batch_size),
  epochs=epochs,
  callbacks=[cp_callback]
)

In [ ]:
accuracy = training_history.history['accuracy']
val_accuracy = training_history.history['val_accuracy']

loss = training_history.history['loss']
val_loss = training_history.history['val_loss']

epochs_range = range(20)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, accuracy, label='Training Accuracy')
plt.plot(epochs_range, val_accuracy, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()